In [ ]:
import requests
import pymongo
import os
from dotenv import load_dotenv

load_dotenv()

# --- MongoDB connection ---
client = pymongo.MongoClient(os.getenv("MONGO_URI"))
db = client["sports_db"]

# --- API key ---
API_KEY = os.getenv("API_KEY")
headers = {"x-apisports-key": API_KEY}

# client.drop_database("sports_db")

# --- Helper function to fetch & store ---
def fetch_and_store(base_url, endpoints, sport_name):
    for endpoint, params in endpoints.items():
        print(f"Fetching {sport_name} -> {endpoint}")
        url = f"{base_url}/{endpoint}"
        response = requests.get(url, headers=headers, params=params).json()

        if "response" in response:
            if len(response["response"]) > 0:
                collection_name = f"{sport_name}_{endpoint}"
                db[collection_name].delete_many({})  # Clear old data
                db[collection_name].insert_many(response["response"])
                print(f"Inserted {len(response['response'])} docs into {collection_name}")
            else:
                print(f"No data for {sport_name} -> {endpoint}")
        else:
            print(f"Error fetching {sport_name} -> {endpoint}: {response}")


# --- Soccer (Premier League 2023 example) ---
soccer_base = "https://v3.football.api-sports.io"
soccer_endpoints = {
    "leagues": {},
    "teams": {"league": 39, "season": 2023},
    "players": {"league": 39, "season": 2023},
    "fixtures": {"league": 39, "season": 2023},
}
fetch_and_store(soccer_base, soccer_endpoints, "soccer")

# --- Basketball (NBA 2023/24 example) ---
basket_base = "https://v1.basketball.api-sports.io"
basket_endpoints = {
    "leagues": {},
    "teams": {"league": 12, "season": "2023-2024"},
    "players": {"league": 12, "season": "2023-2024"},
    "games": {"league": 12, "season": "2023-2024"},
}
fetch_and_store(basket_base, basket_endpoints, "basketball")

# --- Formula 1 (2023 season example) ---
f1_base = "https://v1.formula-1.api-sports.io"
f1_endpoints = {
    "competitions": {},
    "drivers": {"season": 2023},
    "teams": {"season": 2023},
    "races": {"season": 2023},
    "rankings/drivers": {"season": 2023},
    "rankings/teams": {"season": 2023},
}
fetch_and_store(f1_base, f1_endpoints, "f1")


In [ ]:

# Soccer DDL, ya luego de haber creado los diagramas con dbdiagram, es prácticamente el mismo código

ddl_statements_soccer = """
CREATE TABLE dim_time (
    time_key SERIAL PRIMARY KEY,
    date_date DATE UNIQUE,
    year INT,
    month INT,
    day INT,
    weekday VARCHAR,
    is_weekend BOOLEAN,
    hour INT
);

CREATE TABLE dim_team (
    team_key SERIAL PRIMARY KEY,
    api_team_id BIGINT UNIQUE,
    name TEXT,
    country TEXT,
    founded INT,
    stadium_name TEXT,
    city TEXT,
    short_code VARCHAR
);

CREATE TABLE dim_league (
    league_key SERIAL PRIMARY KEY,
    api_league_id BIGINT UNIQUE,
    name TEXT,
    country TEXT,
    season VARCHAR
);

CREATE TABLE dim_venue (
    venue_key SERIAL PRIMARY KEY,
    api_venue_id BIGINT UNIQUE,
    name TEXT,
    city TEXT,
    capacity INT
);

CREATE TABLE dim_referee (
    referee_key SERIAL PRIMARY KEY,
    api_referee_id BIGINT UNIQUE,
    name TEXT,
    nationality TEXT
);

CREATE TABLE fact_match (
    match_key SERIAL PRIMARY KEY,
    api_match_id BIGINT UNIQUE,
    league_key INT REFERENCES dim_league(league_key),
    season VARCHAR,
    time_key INT REFERENCES dim_time(time_key),
    venue_key INT REFERENCES dim_venue(venue_key),
    referee_key INT REFERENCES dim_referee(referee_key),
    home_team_key INT REFERENCES dim_team(team_key),
    away_team_key INT REFERENCES dim_team(team_key),
    home_goals INT,
    away_goals INT,
    attendance INT,
    possession_home NUMERIC,
    possession_away NUMERIC,
    shots_home INT,
    shots_away INT,
    yellow_cards_home INT,
    yellow_cards_away INT,
    red_cards_home INT,
    red_cards_away INT
);
"""

In [ ]:
ddl_statements_basket = """CREATE TABLE dim_team_basketball (
    id SERIAL PRIMARY KEY,
    api_team_id BIGINT,
    name VARCHAR,
    city VARCHAR
);

CREATE TABLE dim_player_basketball (
    id SERIAL PRIMARY KEY,
    api_player_id BIGINT,
    full_name VARCHAR,
    position VARCHAR,
    nationality VARCHAR,
    birthdate DATE
);

CREATE TABLE dim_league_basketball (
    id SERIAL PRIMARY KEY,
    api_league_id BIGINT UNIQUE,
    name VARCHAR,
    country VARCHAR
);

CREATE TABLE dim_date (
    id SERIAL PRIMARY KEY,
    date DATE,
    year INT,
    month INT,
    day INT,
    day_of_week VARCHAR
);

CREATE TABLE fact_game_basketball (
    id SERIAL PRIMARY KEY,
    date_id INT REFERENCES dim_date(id),
    team_home_id INT REFERENCES dim_team_basketball(id),
    team_away_id INT REFERENCES dim_team_basketball(id),
    player_id INT REFERENCES dim_player_basketball(id),
    league_id INT REFERENCES dim_league_basketball(id),
    points INT,
    rebounds INT,
    assists INT,
    steals INT,
    blocks INT
);
"""

In [12]:
ddl_statements_f1 = """
CREATE TABLE dim_driver (
    driver_id SERIAL PRIMARY KEY,
    api_driver_id INT UNIQUE,
    driver_name VARCHAR,
    birthdate DATE,
    nationality VARCHAR,
    number INT
);

CREATE TABLE dim_team_f1 (
    team_id SERIAL PRIMARY KEY,
    api_team_id INT UNIQUE,
    team_name VARCHAR,
    base VARCHAR,
    principal VARCHAR
);

CREATE TABLE dim_race (
    race_id SERIAL PRIMARY KEY,
    api_race_id INT UNIQUE,
    season INT,
    round INT,
    race_name VARCHAR,
    date DATE
);

CREATE TABLE dim_circuit (
    circuit_id SERIAL PRIMARY KEY,
    api_circuit_id INT UNIQUE,
    circuit_name VARCHAR,
    location VARCHAR,
    country VARCHAR,
    length_km FLOAT
);

CREATE TABLE fact_race_results (
    race_result_id SERIAL PRIMARY KEY,
    driver_id INT REFERENCES dim_driver(driver_id),
    team_id INT REFERENCES dim_team_f1(team_id),
    race_id INT REFERENCES dim_race(race_id),
    circuit_id INT REFERENCES dim_circuit(circuit_id),
    position INT,
    points FLOAT,
    laps INT,
    time VARCHAR,
    status VARCHAR
);
"""

In [20]:
st = """
ALTER TABLE dim_date
ADD CONSTRAINT dim_date_date_unique UNIQUE (date);


"""

In [21]:
%pip install psycopg2-binary
import psycopg2
import os
from dotenv import load_dotenv

def create_schema(ddl_statements):
    # Ajusta estos valores con los de tu base de datos
    conn = psycopg2.connect(
        dbname=os.getenv("PG_DB"),
        user=os.getenv("PG_USER"),
        password=os.getenv("PG_PASS"),
        host=os.getenv("PG_HOST"),
        port=os.getenv("PG_PORT")
    )
    conn.autocommit = True
    cursor = conn.cursor()

    try:
        cursor.execute(ddl_statements)
        print("✅ Tablas creadas exitosamente")
    except Exception as e:
        print("⚠️ Error creando tablas:", e)
    finally:
        cursor.close()
        conn.close()

if __name__ == "__main__":
    load_dotenv()
    create_schema(st)

Note: you may need to restart the kernel to use updated packages.
✅ Tablas creadas exitosamente
